In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col

In [29]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("DDAM Project") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

23/12/04 15:12:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


SIMPLE LINEAR REGRESSION: NO MISSING VALUES

In [30]:
df_player = spark.read.csv("/Users/davidepiccoli/Desktop/ddam/DDAM project/Users/davidepiccoli/Desktop/ddam/DDAM project/df_playerInfo/part-00000-9f6beb77-af0c-41f3-a40c-5a5cfb39e71f-c000.csv", header=True, inferSchema=True)

In [31]:
df_player.show(10)

+--------------------+---+----+-----+
|          playerName|Pos|  Ht|   Wt|
+--------------------+---+----+-----+
|      Alaa Abdelnaby|F-C|6-10|240.0|
|     Zaid Abdul-Aziz|C-F| 6-9|235.0|
|Kareem Abdul-Jabbar*|  C| 7-2|225.0|
|  Mahmoud Abdul-Rauf|  G| 6-1|162.0|
|   Tariq Abdul-Wahad|  F| 6-6|223.0|
| Shareef Abdur-Rahim|  F| 6-9|225.0|
|       Tom Abernethy|  F| 6-7|220.0|
|         Forest Able|  G| 6-3|180.0|
|      John Abramovic|  F| 6-3|195.0|
|        Álex Abrines|G-F| 6-6|200.0|
+--------------------+---+----+-----+
only showing top 10 rows



In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Create a Spark session
spark = SparkSession.builder.appName("Feet to Meters Conversion").getOrCreate()

# Assuming you have a DataFrame named 'your_dataframe' with a column 'Ht' in the format "feet-inches"
# Replace 'your_dataframe' with the actual name of your DataFrame

# Split the "Ht" column into feet and inches
split_col = expr("split(Ht, '-')")
df_player = df_player.withColumn("feet", split_col.getItem(0).cast("double"))
df_player = df_player.withColumn("inches", split_col.getItem(1).cast("double"))

# Perform the conversion and overwrite the "Ht" column
conversion_expr = expr("feet * 30.48 + inches * 2.54")
df_player = df_player.withColumn("Ht", conversion_expr)

# Show the result
df_player.show()


23/12/04 15:12:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+---+------------------+-----+----+------+
|          playerName|Pos|                Ht|   Wt|feet|inches|
+--------------------+---+------------------+-----+----+------+
|      Alaa Abdelnaby|F-C|            208.28|240.0| 6.0|  10.0|
|     Zaid Abdul-Aziz|C-F|            205.74|235.0| 6.0|   9.0|
|Kareem Abdul-Jabbar*|  C|218.44000000000003|225.0| 7.0|   2.0|
|  Mahmoud Abdul-Rauf|  G|            185.42|162.0| 6.0|   1.0|
|   Tariq Abdul-Wahad|  F|            198.12|223.0| 6.0|   6.0|
| Shareef Abdur-Rahim|  F|            205.74|225.0| 6.0|   9.0|
|       Tom Abernethy|  F|            200.66|220.0| 6.0|   7.0|
|         Forest Able|  G|             190.5|180.0| 6.0|   3.0|
|      John Abramovic|  F|             190.5|195.0| 6.0|   3.0|
|        Álex Abrines|G-F|            198.12|200.0| 6.0|   6.0|
|    Precious Achiuwa|  F|             203.2|225.0| 6.0|   8.0|
|          Alex Acker|  G|195.57999999999998|185.0| 6.0|   5.0|
|        Don Ackerman|  G|            18

In [33]:
from pyspark.sql.functions import col,isnan, when, count
df_player.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_player.columns]
   ).show()

+----------+---+---+---+----+------+
|playerName|Pos| Ht| Wt|feet|inches|
+----------+---+---+---+----+------+
|         0|  0|  0|  5|   0|     0|
+----------+---+---+---+----+------+



In [34]:
# Drop rows with missing values in the "Wt" column
df_player = df_player.na.drop(subset=["Wt"])

# Show the resulting DataFrame
df_player.show()

+--------------------+---+------------------+-----+----+------+
|          playerName|Pos|                Ht|   Wt|feet|inches|
+--------------------+---+------------------+-----+----+------+
|      Alaa Abdelnaby|F-C|            208.28|240.0| 6.0|  10.0|
|     Zaid Abdul-Aziz|C-F|            205.74|235.0| 6.0|   9.0|
|Kareem Abdul-Jabbar*|  C|218.44000000000003|225.0| 7.0|   2.0|
|  Mahmoud Abdul-Rauf|  G|            185.42|162.0| 6.0|   1.0|
|   Tariq Abdul-Wahad|  F|            198.12|223.0| 6.0|   6.0|
| Shareef Abdur-Rahim|  F|            205.74|225.0| 6.0|   9.0|
|       Tom Abernethy|  F|            200.66|220.0| 6.0|   7.0|
|         Forest Able|  G|             190.5|180.0| 6.0|   3.0|
|      John Abramovic|  F|             190.5|195.0| 6.0|   3.0|
|        Álex Abrines|G-F|            198.12|200.0| 6.0|   6.0|
|    Precious Achiuwa|  F|             203.2|225.0| 6.0|   8.0|
|          Alex Acker|  G|195.57999999999998|185.0| 6.0|   5.0|
|        Don Ackerman|  G|            18

In [35]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline


# Step 1: Prepare the features and label columns
feature_columns = ["Ht"]
label_column = "Wt"

# Step 2: Create a VectorAssembler to assemble the features into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 3: Create a LinearRegression model
linear_regression = LinearRegression(featuresCol="features", labelCol=label_column)

# Step 4: Create a pipeline to chain the VectorAssembler and LinearRegression stages
pipeline = Pipeline(stages=[vector_assembler, linear_regression])

# Step 5: Split the data into training and testing sets
train_data, test_data = df_player.randomSplit([0.8, 0.2], seed=42)

# Step 6: Fit the pipeline on the training data
model = pipeline.fit(train_data)

# Step 7: Make predictions on the testing data
predictions = model.transform(test_data)

# Step 8: Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Step 9: Print the root mean squared error
print("Root Mean Squared Error (RMSE) on test data =", rmse)

# Optionally, you can inspect the coefficients and intercept of the linear regression model
coefficients = model.stages[-1].coefficients
intercept = model.stages[-1].intercept

print("Coefficients:", coefficients)
print("Intercept:", intercept)


23/12/04 15:12:55 WARN Instrumentation: [3a35fc20] regParam is zero, which might cause numerical instability and overfitting.


Root Mean Squared Error (RMSE) on test data = 14.902391582358792
Coefficients: [2.320729614006565]
Intercept: -250.31796750672592


In [36]:
from pyspark.ml.evaluation import RegressionEvaluator

# Step 1: Create a RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="Wt", predictionCol="prediction", metricName="r2")

# Step 2: Evaluate the model on the testing data
r2 = evaluator.evaluate(predictions)

# Step 3: Print the R-squared value
print("R-squared:", r2)


R-squared: 0.6666476008737623


RIDGE REGULARIZATION

In [37]:
# Step 1: Prepare the features and label columns
feature_columns = ["Ht"]
label_column = "Wt"

# Step 2: Create a VectorAssembler to assemble the features into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 3: Create a LinearRegression model with regularization
# Use elasticNetParam to control the combination of L1 and L2 regularization
# For L1 regularization (Lasso), set elasticNetParam = 1
# For L2 regularization (Ridge), set elasticNetParam = 0
linear_regression = LinearRegression(featuresCol="features", labelCol=label_column, elasticNetParam=0.0)

# Step 4: Create a pipeline to chain the VectorAssembler and LinearRegression stages
pipeline = Pipeline(stages=[vector_assembler, linear_regression])

# Step 5: Split the data into training and testing sets
train_data, test_data = df_player.randomSplit([0.8, 0.2], seed=42)

# Step 6: Fit the pipeline on the training data
model = pipeline.fit(train_data)

# Step 7: Make predictions on the testing data
predictions = model.transform(test_data)

# Step 8: Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Step 9: Print the root mean squared error
print("Root Mean Squared Error (RMSE) on test data =", rmse)

# Optionally, you can inspect the coefficients and intercept of the linear regression model
coefficients = model.stages[-1].coefficients
intercept = model.stages[-1].intercept

print("Coefficients:", coefficients)
print("Intercept:", intercept)


23/12/04 15:12:57 WARN Instrumentation: [0b54c06a] regParam is zero, which might cause numerical instability and overfitting.


Root Mean Squared Error (RMSE) on test data = 14.902391582358792
Coefficients: [2.320729614006565]
Intercept: -250.31796750672592


In [38]:
# Step 1: Create a RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="Wt", predictionCol="prediction", metricName="r2")

# Step 2: Evaluate the model on the testing data
r2 = evaluator.evaluate(predictions)

# Step 3: Print the R-squared value
print("R-squared:", r2)

R-squared: 0.6666476008737623


LASSO REGULARIZATION

In [39]:
# Step 1: Prepare the features and label columns
feature_columns = ["Ht"]
label_column = "Wt"

# Step 2: Create a VectorAssembler to assemble the features into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 3: Create a LinearRegression model with regularization
# Use elasticNetParam to control the combination of L1 and L2 regularization
# For L1 regularization (Lasso), set elasticNetParam = 1
# For L2 regularization (Ridge), set elasticNetParam = 0
linear_regression = LinearRegression(featuresCol="features", labelCol=label_column, elasticNetParam=1.0)

# Step 4: Create a pipeline to chain the VectorAssembler and LinearRegression stages
pipeline = Pipeline(stages=[vector_assembler, linear_regression])

# Step 5: Split the data into training and testing sets
train_data, test_data = df_player.randomSplit([0.8, 0.2], seed=42)

# Step 6: Fit the pipeline on the training data
model = pipeline.fit(train_data)

# Step 7: Make predictions on the testing data
predictions = model.transform(test_data)

# Step 8: Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol=label_column, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Step 9: Print the root mean squared error
print("Root Mean Squared Error (RMSE) on test data =", rmse)

# Optionally, you can inspect the coefficients and intercept of the linear regression model
coefficients = model.stages[-1].coefficients
intercept = model.stages[-1].intercept

print("Coefficients:", coefficients)
print("Intercept:", intercept)


23/12/04 15:12:58 WARN Instrumentation: [bd46e1d3] regParam is zero, which might cause numerical instability and overfitting.


Root Mean Squared Error (RMSE) on test data = 14.902391582358792
Coefficients: [2.320729614006565]
Intercept: -250.31796750672592


In [40]:
# Step 1: Create a RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="Wt", predictionCol="prediction", metricName="r2")

# Step 2: Evaluate the model on the testing data
r2 = evaluator.evaluate(predictions)

# Step 3: Print the R-squared value
print("R-squared:", r2)

R-squared: 0.6666476008737623


In [41]:
predictions.collect()

[Row(playerName='A.J. English', Pos='G', Ht=190.5, Wt=175.0, feet=6.0, inches=3.0, features=DenseVector([190.5]), prediction=191.78102396152477),
 Row(playerName='A.J. Wynder', Pos='G', Ht=187.96, Wt=180.0, feet=6.0, inches=2.0, features=DenseVector([187.96]), prediction=185.88637074194807),
 Row(playerName='Aaron Brooks', Pos='G', Ht=182.88, Wt=161.0, feet=6.0, inches=0.0, features=DenseVector([182.88]), prediction=174.09706430279473),
 Row(playerName='Aaron Holiday', Pos='G', Ht=182.88, Wt=185.0, feet=6.0, inches=0.0, features=DenseVector([182.88]), prediction=174.09706430279473),
 Row(playerName='Aaron Swinson', Pos='F', Ht=195.57999999999998, Wt=230.0, feet=6.0, inches=5.0, features=DenseVector([195.58]), prediction=203.57033040067805),
 Row(playerName='Abdul Jeelani', Pos='F-C', Ht=203.2, Wt=210.0, feet=6.0, inches=8.0, features=DenseVector([203.2]), prediction=221.25429005940808),
 Row(playerName='Adam Morrison', Pos='F', Ht=203.2, Wt=205.0, feet=6.0, inches=8.0, features=DenseVe

- check for missing values and put them in the test set
- try to implement multiple linear regression (?)
- 

SIMPLE LINEAR REGRESSION: WITH MISSING VALUES

In [42]:
df_player1 = spark.read.csv("/Users/davidepiccoli/Desktop/ddam/DDAM project/Users/davidepiccoli/Desktop/ddam/DDAM project/df_playerInfo/part-00000-9f6beb77-af0c-41f3-a40c-5a5cfb39e71f-c000.csv", header=True, inferSchema=True)

In [43]:
df_player1.show(10)

+--------------------+---+----+-----+
|          playerName|Pos|  Ht|   Wt|
+--------------------+---+----+-----+
|      Alaa Abdelnaby|F-C|6-10|240.0|
|     Zaid Abdul-Aziz|C-F| 6-9|235.0|
|Kareem Abdul-Jabbar*|  C| 7-2|225.0|
|  Mahmoud Abdul-Rauf|  G| 6-1|162.0|
|   Tariq Abdul-Wahad|  F| 6-6|223.0|
| Shareef Abdur-Rahim|  F| 6-9|225.0|
|       Tom Abernethy|  F| 6-7|220.0|
|         Forest Able|  G| 6-3|180.0|
|      John Abramovic|  F| 6-3|195.0|
|        Álex Abrines|G-F| 6-6|200.0|
+--------------------+---+----+-----+
only showing top 10 rows



In [44]:
# Create a Spark session
spark = SparkSession.builder.appName("Feet to Meters Conversion").getOrCreate()

# Assuming you have a DataFrame named 'your_dataframe' with a column 'Ht' in the format "feet-inches"
# Replace 'your_dataframe' with the actual name of your DataFrame

# Split the "Ht" column into feet and inches
split_col = expr("split(Ht, '-')")
df_player1 = df_player1.withColumn("feet", split_col.getItem(0).cast("double"))
df_player1 = df_player1.withColumn("inches", split_col.getItem(1).cast("double"))

# Perform the conversion and overwrite the "Ht" column
conversion_expr = expr("feet * 30.48 + inches * 2.54")
df_player1 = df_player1.withColumn("Ht", conversion_expr)

# Show the result
df_player1.show()

+--------------------+---+------------------+-----+----+------+
|          playerName|Pos|                Ht|   Wt|feet|inches|
+--------------------+---+------------------+-----+----+------+
|      Alaa Abdelnaby|F-C|            208.28|240.0| 6.0|  10.0|
|     Zaid Abdul-Aziz|C-F|            205.74|235.0| 6.0|   9.0|
|Kareem Abdul-Jabbar*|  C|218.44000000000003|225.0| 7.0|   2.0|
|  Mahmoud Abdul-Rauf|  G|            185.42|162.0| 6.0|   1.0|
|   Tariq Abdul-Wahad|  F|            198.12|223.0| 6.0|   6.0|
| Shareef Abdur-Rahim|  F|            205.74|225.0| 6.0|   9.0|
|       Tom Abernethy|  F|            200.66|220.0| 6.0|   7.0|
|         Forest Able|  G|             190.5|180.0| 6.0|   3.0|
|      John Abramovic|  F|             190.5|195.0| 6.0|   3.0|
|        Álex Abrines|G-F|            198.12|200.0| 6.0|   6.0|
|    Precious Achiuwa|  F|             203.2|225.0| 6.0|   8.0|
|          Alex Acker|  G|195.57999999999998|185.0| 6.0|   5.0|
|        Don Ackerman|  G|            18

In [45]:
from pyspark.sql.functions import col,isnan, when, count
df_player.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_player.columns]
   ).show()

+----------+---+---+---+----+------+
|playerName|Pos| Ht| Wt|feet|inches|
+----------+---+---+---+----+------+
|         0|  0|  0|  0|   0|     0|
+----------+---+---+---+----+------+



In [46]:

# Create a Spark session
spark = SparkSession.builder.appName("Missing Values Test Set").getOrCreate()

# Assuming you have a DataFrame named 'your_dataframe'
# Replace 'your_dataframe' with the actual name of your DataFrame

# Identify records with missing values
missing_values_df = df_player1.filter(col("Wt").isNull())

# Sample 5 records from the missing values DataFrame
test_set = missing_values_df

# Show the test set
test_set.show()


+---------------+---+------+----+----+------+
|     playerName|Pos|    Ht|  Wt|feet|inches|
+---------------+---+------+----+----+------+
|       Dick Lee|  F|198.12|NULL| 6.0|   6.0|
|Murray Mitchell|  C|198.12|NULL| 6.0|   6.0|
|     Paul Nolen|  C|208.28|NULL| 6.0|  10.0|
|     Ray Wertis|  G|180.34|NULL| 5.0|  11.0|
|       Bob Wood|  G| 177.8|NULL| 5.0|  10.0|
+---------------+---+------+----+----+------+



23/12/04 15:13:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [47]:
# Drop rows with missing values in the "Wt" column
train_set = df_player1.na.drop(subset=["Wt"])

# Show the resulting DataFrame
train_set.show()

+--------------------+---+------------------+-----+----+------+
|          playerName|Pos|                Ht|   Wt|feet|inches|
+--------------------+---+------------------+-----+----+------+
|      Alaa Abdelnaby|F-C|            208.28|240.0| 6.0|  10.0|
|     Zaid Abdul-Aziz|C-F|            205.74|235.0| 6.0|   9.0|
|Kareem Abdul-Jabbar*|  C|218.44000000000003|225.0| 7.0|   2.0|
|  Mahmoud Abdul-Rauf|  G|            185.42|162.0| 6.0|   1.0|
|   Tariq Abdul-Wahad|  F|            198.12|223.0| 6.0|   6.0|
| Shareef Abdur-Rahim|  F|            205.74|225.0| 6.0|   9.0|
|       Tom Abernethy|  F|            200.66|220.0| 6.0|   7.0|
|         Forest Able|  G|             190.5|180.0| 6.0|   3.0|
|      John Abramovic|  F|             190.5|195.0| 6.0|   3.0|
|        Álex Abrines|G-F|            198.12|200.0| 6.0|   6.0|
|    Precious Achiuwa|  F|             203.2|225.0| 6.0|   8.0|
|          Alex Acker|  G|195.57999999999998|185.0| 6.0|   5.0|
|        Don Ackerman|  G|            18

In [49]:
# Step 1: Prepare the features and label columns
feature_columns = ["Ht"]
label_column = "Wt"

# Step 2: Create a VectorAssembler to assemble the features into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Step 3: Create a LinearRegression model
linear_regression = LinearRegression(featuresCol="features", labelCol=label_column)

# Step 4: Create a pipeline to chain the VectorAssembler and LinearRegression stages
pipeline = Pipeline(stages=[vector_assembler, linear_regression])


# Step 5: Fit the pipeline on the training data
model = pipeline.fit(train_set)

# Step 6: Make predictions on the testing data
predictions = model.transform(test_set)

23/12/04 15:14:17 WARN Instrumentation: [0daf10e1] regParam is zero, which might cause numerical instability and overfitting.


In [50]:
predictions.collect()

[Row(playerName='Dick Lee', Pos='F', Ht=198.12, Wt=None, feet=6.0, inches=6.0, features=DenseVector([198.12]), prediction=209.37916596965968),
 Row(playerName='Murray Mitchell', Pos='C', Ht=198.12, Wt=None, feet=6.0, inches=6.0, features=DenseVector([198.12]), prediction=209.37916596965968),
 Row(playerName='Paul Nolen', Pos='C', Ht=208.28, Wt=None, feet=6.0, inches=10.0, features=DenseVector([208.28]), prediction=232.94241645925237),
 Row(playerName='Ray Wertis', Pos='G', Ht=180.34, Wt=None, feet=5.0, inches=11.0, features=DenseVector([180.34]), prediction=168.14347761287243),
 Row(playerName='Bob Wood', Pos='G', Ht=177.8, Wt=None, feet=5.0, inches=10.0, features=DenseVector([177.8]), prediction=162.2526649904743)]